In [ ]:
#package imports
import os
import pandas as pd
import numpy as np
from xml.dom import minidom
import shutil
import sys



#enter the input folder
os.chdir('input')

#read the csv
df = pd.read_csv(r"metadata.csv")

#make the temporary output folder
os.mkdir('output')


#XML Tag Blocks

def titletag():
    #create the title tag
    if type(row['dc.title'])!=float:
        dctitle = root.createElement('dcvalue')
        dctitle.setAttribute('element', 'title')
        dctitle.setAttribute('qualifier', 'none')
        text=root.createTextNode(row['dc.title'])
        dctitle.appendChild(text)
        xml.appendChild(dctitle)

def typetag():
    #create the type tag
    if type(row['dc.type'])!=float:
        dctype = root.createElement('dcvalue')
        dctype.setAttribute('element', 'type')
        dctype.setAttribute('qualifier', 'none')
        text=root.createTextNode(row['dc.type'])
        dctype.appendChild(text)
        xml.appendChild(dctype)
 
def subjecttag():
    #create the subject tag (woooooooo!!!!!) Separate subjects with #
    if type(row['dc.subject'])!=float:
        subject=row['dc.subject'].split('#')
        for s in subject:
            dcsubject = root.createElement('dcvalue')
            dcsubject.setAttribute('element', 'subject')
            dcsubject.setAttribute('qualifier', 'none')
            text=root.createTextNode(s)
            dcsubject.appendChild(text)
            xml.appendChild(dcsubject)

def publishertag():
    #create the publisher tag
    if type(row['dc.publisher'])!=float:
        dcpublisher = root.createElement('dcvalue')
        dcpublisher.setAttribute('element', 'publisher')
        dcpublisher.setAttribute('qualifier', 'none')
        text=root.createTextNode(row['dc.publisher'])
        dcpublisher.appendChild(text)
        xml.appendChild(dcpublisher)

def languagetag():
    #create the language tag
    if type(row['dc.language.iso'])!=float:
        dclanguage = root.createElement('dcvalue')
        dclanguage.setAttribute('element', 'language')
        dclanguage.setAttribute('qualifier', 'iso')
        text=root.createTextNode(row['dc.language.iso'])
        dclanguage.appendChild(text)
        xml.appendChild(dclanguage)

def descriptiontag():    
    #create the description tag
    if type(row['dc.description.abstract'])!=float:
        dcdescription = root.createElement('dcvalue')
        dcdescription.setAttribute('element', 'description')
        dcdescription.setAttribute('qualifier', 'abstract')
        text=root.createTextNode(row['dc.description.abstract'])
        dcdescription.appendChild(text)
        xml.appendChild(dcdescription)

def datetag():        
    #create the date tag
    if type(row['dc.date.issued'])!=float:
        dcdate = root.createElement('dcvalue')
        dcdate.setAttribute('element', 'date')
        dcdate.setAttribute('qualifier', 'issued')
        text=root.createTextNode(row['dc.date.issued'])
        dcdate.appendChild(text)
        xml.appendChild(dcdate)

#Iterate through the metada csv
for n, row in df.iterrows():

    #Make directory for each item
    os.mkdir(row['dc.title'])
    print(row['dc.title']+' folder created')
    
    #copy file into item's directory
    shutil.copy(row['filename'], os.path.join(row['dc.title'], row['filename']))
    print(row['dc.title']+' copied into folder')
    
    #enter the item directory
    os.chdir(row['dc.title'])
    
    #create the xml file
    root = minidom.Document()    
    
    #create the dublin core schema tag
    xml = root.createElement('dublin_core') 
    root.appendChild(xml)
    
    #Run the tag blocks
    titletag()
    typetag()
    subjecttag()
    publishertag()
    languagetag()
    descriptiontag()
    datetag()
    
    #make the xml pretty (for troubleshooting purposes)
    xml_str = root.toprettyxml(indent ="\t") 
    
    #save the xml file
    save_path_file = ("dublin_core.xml")
    with open(save_path_file, "w") as f:
        f.write(xml_str)
    print(row['dc.title']+' XML file created')
    
    #Create the contents file. 
    f = open(("contents"), "a")
    if type(row['description'])!=float:
        f.write(row['filename']+'\t'+'description:'+row['description'])
    else:
        f.write(row['filename']+'\t'+'bundle:ORIGINAL')
    f.close()
    print(row['dc.title']+' contents file created')
    
    #Return to the inputs level
    os.chdir('..')
    
    #move the completed item folders to output
    shutil.move(row['dc.title'], os.path.join('output',row['dc.title']))
    print(row['dc.title']+' folder moved to output folder')

#Make the zip file 
print('Generating zip file')
shutil.make_archive('output', 'zip', 'output')
print('Zip file created')

#Remove the temporary output file (commmented out for testing purposes)
shutil.rmtree('output')
print('Output folder deleted')

#return to the original level
os.chdir('..')

#Print operation successful
print('Operation successful. Please check your input folder for the zip file')
